In [2]:
import pandas as pd
df = pd.read_csv('Electricity_202502232316.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True) - pd.Timedelta(minutes=30)




In [3]:
from data_preparation import pepare_data, preparation_data_darts
df = pepare_data(df)
series_scaled, cov_scaled, sc_tgt, sc_cov = preparation_data_darts(df)
train_size = 0.8

train, val = series_scaled.split_after(train_size)
train_cov, val_cov = cov_scaled.split_after(train_size)

                  Timestamp  Sun_Angle  Sun_Angle_Trend  hour_sin  hour_cos  \
0 2022-11-23 23:00:00+00:00        0.0               -1  0.000000  1.000000   
1 2022-11-24 00:00:00+00:00        0.0               -1  0.258819  0.965926   
2 2022-11-24 01:00:00+00:00        0.0               -1  0.500000  0.866025   

     wd_sin    wd_cos  
0  0.433884 -0.900969  
1  0.433884 -0.900969  
2  0.433884 -0.900969  
Index(['Timestamp', 'Sun_Angle', 'Sun_Angle_Trend', 'hour_sin', 'hour_cos',
       'wd_sin', 'wd_cos'],
      dtype='str')
Timestamp          datetime64[us, UTC]
Sun_Angle                      float64
Sun_Angle_Trend                  int64
hour_sin                       float64
hour_cos                       float64
wd_sin                         float64
wd_cos                         float64
dtype: object
index name: None


In [4]:
INPUT_LEN   = 48   # look-back window
OUTPUT_LEN  = 24   # forecast horizon
BATCH_SIZE  = 256
EPOCHS      = 300
LR          = 1e-3
SEED        = 42   # reproducibility

In [5]:
from darts.models import TiDEModel
model =TiDEModel(
    input_chunk_length   = INPUT_LEN,        # encoder length
    output_chunk_length  = OUTPUT_LEN,        # forecast horizon
    hidden_size          = 256,       # width of every FC layer
    num_encoder_layers   = 2,
    num_decoder_layers   = 2,
    dropout              = 0.0,       # keep dropout kwarg
    batch_size           = BATCH_SIZE,
    random_state         = SEED,
    pl_trainer_kwargs    = dict(
        max_epochs  = EPOCHS,
        accelerator = "cpu",
    ))

Support for PyTorch based likelihood models not available. To enable them, install "darts[torch]" or "darts[all]" (with pip); or "u8darts-torch" or "u8darts-all" (with conda).
Support for Torch based models not available. To enable them, install "darts[torch]" or "darts[all]" (with pip); or "u8darts-torch" or "u8darts-all" (with conda).
The StatsForecast module could not be imported. To enable support for the AutoARIMA, AutoETS and Croston models, please consider installing it.
The `XGBoost` module could not be imported. To enable XGBoost support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md
The `XGBoost` module could not be imported. To enable XGBoost support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md
ImportError: The `(Py)Torch` module could not be imported. To enable (Py)Torch support in Darts, follow the detailed instruct

ImportError: The `(Py)Torch` module could not be imported. To enable (Py)Torch support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md

In [6]:
from darts.metrics import mae
from darts import TimeSeries
import numpy as np

model.fit(  series               = train,   # optional but helps
future_covariates    = train_cov, # <- **required**
val_series           = val,
val_future_covariates= val_cov,
past_covariates= train_cov,
val_past_covariates= val_cov)
  


    # ❻  multi-step forecast matching the full validation span
pred = model.predict(n=len(val)-INPUT_LEN,series= train, past_covariates= TimeSeries.concatenate(train_cov, val_cov),future_covariates=  TimeSeries.concatenate(train_cov, val_cov))

    # ❼  inverse-transform back to original scale
#pred = sc_tgt.inverse_transform(pred)
#truth = sc_tgt.inverse_transform(val)
    
score_df =mae(val, pred)
print(score_df)
    


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

   | Name                  | Type             | Params | Mode 
--------------------------------------------------------------------
0  | criterion             | MSELoss          | 0      | train
1  | train_criterion       | MSELoss          | 0      | train
2  | val_criterion         | MSELoss          | 0      | train
3  | train_metrics         | MetricCollection | 0      | train
4  | val_metrics           | MetricCollection | 0      | train
5  | past_cov_projection   | _ResidualBlock   | 2.8 K  | train
6  | future_cov_projection | _ResidualBlock   | 2.8 K  | train
7  | encoders              | Sequential       | 534 K  | train
8  | decoders              | Sequential       | 460 K  | train
9  | temporal_decoder      | _ResidualBlock   | 726    | train
10 | lookback_skip         | Linear           | 1.2 K  | train
---------------------------------------------------------------

/home/gerbrand/PycharmProjects/TideEnergy/.venv/lib64/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 299: 100%|██████████| 61/61 [00:04<00:00, 12.34it/s, train_loss=806.0, val_loss=3.63e+4]  

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 299: 100%|██████████| 61/61 [00:04<00:00, 12.30it/s, train_loss=806.0, val_loss=3.63e+4]


`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  7.35it/s]
126.80661


In [7]:
import os
import shutil

# Recommended by Darts: use model.save / model.load for full model serialization
MODEL_PATH = '/home/gerbrand/PycharmProjects/models/tide_model'

# Guard: ensure parent exists and remove previous saved model if present
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
if os.path.exists(MODEL_PATH):
    shutil.rmtree(MODEL_PATH)

model.save(MODEL_PATH)
print(f'Model saved to {MODEL_PATH} ✔')


Model saved to /home/gerbrand/PycharmProjects/models/tide_model ✔


In [28]:
ds = model.model.train_sample_shape    # still alive
print(ds)

[(48, 1), (48, 6), (48, 6), (24, 6), None, (24, 1)]
